In [1]:
from matrix_factorisation import nmf_features, transform, transform_2, preprocess, get_static_features

In [2]:
import cvxpy as cvx
import numpy as np
import pandas as pd

In [3]:
import  os

import numpy as np
import pandas as pd
import pickle
from features import feature_map

In [4]:
import  os

import numpy as np
import pandas as pd


from create_df_larger import read_df_larger
df, dfc, all_homes, appliance_min, national_average = read_df_larger()

df = df.rename(columns={'house_num_rooms':'num_rooms',
                        'num_occupants':'total_occupants',
                        'difference_ratio_min_max':'ratio_difference_min_max'})
K_min, K_max = 1,6
F_min, F_max=1,8

from all_functions import *
from features_larger import *

In [5]:
import itertools
feature_combinations = [['None']]
for l in range(1,4):
    for a in itertools.combinations(['occ','area','rooms'], l):
        feature_combinations.append(list(a))

In [ ]:
#out_overall = {}
#error_overall = {}
appliance="fridge"
for constant in [0.01, 0.1, 1, 10, 100]:
#for appliance in ['fridge']:
#for appliance in ['fridge','hvac','mw','wm']:

    out = {}
    error = {}
    print appliance
    if appliance=="hvac":
        start, end = 5,11
    else:
        start, end=1,13
    X_matrix, X_normalised, col_max, col_min, appliance_cols, aggregate_cols = preprocess(df, dfc, appliance)
    static_features= get_static_features(dfc, X_normalised)
    
    
    
    for feature_comb in np.array(feature_combinations)[:2]:
        out[tuple(feature_comb)]={}
        error[tuple(feature_comb)]={}
        
        if 'None' in feature_comb:
            idx_user=None
            data_user=None
        else:
            idx_user = {}
            data_user = {}
            dictionary_static = {}
            for feature in feature_comb:
                dictionary_static[feature]=static_features[feature]
            static_features_df = pd.DataFrame(dictionary_static, index=range(len(X_normalised.index)))


            for fe in static_features_df.columns:
                idx_user[fe]=np.where(static_features_df[fe].notnull())[0]
                data_user[fe]=static_features_df[fe].dropna().values
            
        for lat in range(2,3):
            
            if lat<len(feature_comb):
                continue
            out[tuple(feature_comb)][lat]={}


            for home in X_normalised.index:
                print home

                X_home = X_normalised.copy()
                for month in range(start, end):
                    X_home.loc[home, '%s_%d' %(appliance, month)] = np.NAN
                mask = X_home.notnull().values
                # Ensure repeatably random problem data.
                A = X_home.copy()
                #X, Y, res = nmf_features(A, lat, constant, True, idx_user, data_user, 10)
                X, Y, res = nmf_features(A, lat, constant, False, idx_user, data_user, 10)


                pred_df = pd.DataFrame(Y*X)
                pred_df.columns = X_normalised.columns
                pred_df.index = X_normalised.index
                out[tuple(feature_comb)][lat][home] = transform_2(pred_df.ix[home], appliance, col_max, col_min)[appliance_cols]

            # Evaluation

            gt = X_matrix[appliance_cols]

            pr = pd.DataFrame(out[tuple(feature_comb)][lat]).T

            error_df = (pr-gt).abs()
            error[tuple(feature_comb)][lat] = error_df.div(gt).mul(100).median().mean()
            print error[tuple(feature_comb)][lat], feature_comb, lat
    out_overall[constant]=out
    error_overall[constant]=error

In [8]:
error

{('None',): {}}

In [13]:
cvx.Minimize(cvx.norm(A.values[mask] - (Y*X)[mask], 'fro')).value

7.0987166464351361

In [17]:
pd.DataFrame(Y*X).describe()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,...,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01,5.900000e+01
mean,2.981428e-11,2.586719e-11,2.703074e-11,2.530747e-11,2.366780e-11,2.027923e-11,2.143374e-11,2.307231e-11,2.471422e-11,2.345270e-11,...,2.004286e-09,2.187871e-09,2.543484e-09,3.041165e-09,3.872563e-09,4.180616e-09,3.377031e-09,2.648475e-09,1.864858e-09,1.997312e-09
std,1.614637e-11,1.400876e-11,1.463890e-11,1.370564e-11,1.281765e-11,1.098252e-11,1.160776e-11,1.249515e-11,1.338435e-11,1.270116e-11,...,1.085451e-09,1.184874e-09,1.377461e-09,1.646988e-09,2.097244e-09,2.264075e-09,1.828881e-09,1.434321e-09,1.009942e-09,1.081674e-09
min,8.667420e-12,7.519947e-12,7.858204e-12,7.357228e-12,6.880552e-12,5.895449e-12,6.231080e-12,6.707435e-12,7.184761e-12,6.818021e-12,...,5.826735e-10,6.360441e-10,7.394255e-10,8.841082e-10,1.125807e-09,1.215362e-09,9.817491e-10,7.699478e-10,5.421397e-10,5.806458e-10
25%,1.929457e-11,1.674018e-11,1.749318e-11,1.637795e-11,1.531682e-11,1.312388e-11,1.387103e-11,1.493144e-11,1.599402e-11,1.517762e-11,...,1.297091e-09,1.415900e-09,1.646038e-09,1.968116e-09,2.506163e-09,2.705522e-09,2.185475e-09,1.713983e-09,1.206859e-09,1.292578e-09
50%,2.611114e-11,2.265430e-11,2.367333e-11,2.216410e-11,2.072809e-11,1.776040e-11,1.877151e-11,2.020656e-11,2.164454e-11,2.053971e-11,...,1.755340e-09,1.916122e-09,2.227565e-09,2.663431e-09,3.391563e-09,3.661353e-09,2.957580e-09,2.319515e-09,1.633229e-09,1.749231e-09
75%,3.502905e-11,3.039159e-11,3.175864e-11,2.973396e-11,2.780750e-11,2.382624e-11,2.518268e-11,2.710785e-11,2.903695e-11,2.755478e-11,...,2.354853e-09,2.570548e-09,2.988360e-09,3.573091e-09,4.549907e-09,4.911841e-09,3.967702e-09,3.111715e-09,2.191038e-09,2.346658e-09
max,7.622241e-11,6.613139e-11,6.910607e-11,6.470042e-11,6.050847e-11,5.184535e-11,5.479693e-11,5.898605e-11,6.318372e-11,5.995856e-11,...,5.124106e-09,5.593454e-09,6.502603e-09,7.774963e-09,9.900493e-09,1.068805e-08,8.633629e-09,6.771021e-09,4.767647e-09,5.106274e-09


In [7]:
%matplotlib inline
pd.DataFrame(error).T.min().min()

48.568681550176741

In [7]:
X_matrix.columns

Index([u'fridge_1', u'fridge_2', u'fridge_3', u'fridge_4', u'fridge_5',
       u'fridge_6', u'fridge_7', u'fridge_8', u'fridge_9', u'fridge_10',
       u'fridge_11', u'fridge_12', u'aggregate_1', u'aggregate_2',
       u'aggregate_3', u'aggregate_4', u'aggregate_5', u'aggregate_6',
       u'aggregate_7', u'aggregate_8', u'aggregate_9', u'aggregate_10',
       u'aggregate_11', u'aggregate_12'],
      dtype='object')

In [26]:
feature_combinations

[['None'],
 ['occ'],
 ['area'],
 ['rooms'],
 ['occ', 'area'],
 ['occ', 'rooms'],
 ['area', 'rooms'],
 ['occ', 'area', 'rooms']]